# Implementing VAE

In [1]:
#imports
from tensorflow.keras.layers import Dense, Activation, Add, Multiply, Input, Reshape, Lambda, Concatenate
from tensorflow.keras.models import Model, Sequential
from keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from tensorflow.keras.utils import plot_model
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

#get data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.
x_test = x_test.reshape(-1, 784).astype('float32') / 255.

#x_train = np.expand_dims(x_train, axis=-1)
#x_test = np.expand_dims(x_test, axis=-1)

#set parameters
hidden_layer_size = 512
latent_encoding_size = 3

epochs = 50
batch_size = 128

/home/eva/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/eva/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/eva/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/eva/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarnin

In [4]:
#make the VAE network skeleton

def Sample(x):
    #take half for mu and half for sigma. 
    #and get epsilon from N(0,1).
    batch_size = tf.shape(x)[0]
    latent_dim = tf.shape(x)[1]
    half_latent = tf.floordiv(latent_dim, 2)
    eps = tf.random_normal(shape=(batch_size, half_latent))
    return tf.slice(x, [0,0], [batch_size,half_latent]) + tf.exp(0.5 * tf.slice(x, [0,half_latent], [batch_size,half_latent])) * eps
    #keras.backend.slice(x, start, size)
    
Encoder = Sequential()
Encoder.add(Dense(hidden_layer_size, input_shape = (784,)))
Encoder.add(Activation("relu"))
Encoder.add(Dense(2*latent_encoding_size, name="MuAndSigma"))
Encoder.add(Lambda(Sample, output_shape=(3,)))
                                                              
Decoder = Sequential()
Decoder.add(Dense(hidden_layer_size, input_shape=(latent_encoding_size,)))
Decoder.add(Activation("relu"))
Decoder.add(Dense(784))
Decoder.add(Activation("sigmoid"))

VAE = Model(inputs = Encoder.input, outputs=Decoder(Encoder.output))

VAE.summary()
Encoder.summary()
Decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3_input (InputLayer)   [(None, 784)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
activation_3 (Activation)    (None, 512)               0         
_________________________________________________________________
MuAndSigma (Dense)           (None, 6)                 3078      
_________________________________________________________________
lambda_1 (Lambda)            (None, None)              0         
_________________________________________________________________
sequential_3 (Sequential)    (None, 784)               404240    
Total params: 809,238
Trainable params: 809,238
Non-trainable params: 0
_____________________________________________________

In [5]:
def KL_loss(x_GT, x):
    batch_size = tf.shape(x)[0]
    latent_dim = tf.shape(x)[1]
    half_latent = tf.floordiv(latent_dim, 2)
    mu = tf.slice(x, [0,0], [batch_size,half_latent])
    sigma = tf.slice(x, [0,half_latent], [batch_size,half_latent])
    return tf.reduce_mean(- 0.5 * tf.reduce_sum(1 + sigma - tf.square(mu) - tf.exp(sigma), axis=-1))

def reconstruction_loss(x_GT,x):
    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(x_GT, x) * 784) #vae_model.outputs[0].log_prob(inp), axis=-1)

VAE.add_loss(KL_loss(None, Encoder.get_layer('MuAndSigma').output))
VAE.add_loss(reconstruction_loss(VAE.inputs[0], VAE.outputs[0]))

W0826 12:37:32.226842 140436686231360 deprecation.py:323] From /home/eva/.local/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
#trening
opt = Adam(lr=0.001)

VAE.compile(optimizer=opt, metrics=["accuracy"])

earlyStop = EarlyStopping(monitor='val_loss', min_delta=0.5, patience=15) #, restore_best_weights=True)

hist = VAE.fit(x_train, validation_data=(x_test, None), epochs=epochs, batch_size=batch_size)

W0826 12:37:38.203547 140436686231360 training_utils.py:1101] Output sequential_3 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to sequential_3.


Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 6s 101us/sample - loss: 186.9393 - val_loss: 159.1816
Epoch 2/50
60000/60000 [==============================] - 7s 108us/sample - loss: 156.1878 - val_loss: 153.7721
Epoch 3/50
60000/60000 [==============================] - 6s 102us/sample - loss: 152.7373 - val_loss: 151.5217
Epoch 4/50
60000/60000 [==============================] - 6s 99us/sample - loss: 150.6250 - val_loss: 149.7150
Epoch 5/50
60000/60000 [==============================] - 7s 117us/sample - loss: 148.9948 - val_loss: 148.3116
Epoch 6/50
60000/60000 [==============================] - 6s 103us/sample - loss: 147.6873 - val_loss: 147.2771
Epoch 7/50
60000/60000 [==============================] - 6s 96us/sample - loss: 146.5804 - val_loss: 146.1891
Epoch 8/50
60000/60000 [==============================] - 6s 101us/sample - loss: 145.6648 - val_loss: 145.6292
Epoch 9/50
60000/60000 [==============================] 